In [ ]:
%config Completer.use_jedi = False

import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

# # Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path = 'best-new.pt')



model.half()
model.conf = 0.75
model.iou = 0.5

In [ ]:
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import traitlets
import ipywidgets
import cv2
import numpy as np
import pandas
import time
from jetbot import Robot

In [ ]:
# Displaying camera output

camera = Camera.instance(width=300, height=300, fps= 3)


image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

In [ ]:
robot = Robot()

In [ ]:
obstacle_detected = False
motor_values = []
forward_counter = 0
forward_delay = 10


def detect(img, model):
    global obstacle_detected, motor_values, forward_counter
    classes = ['obstacle']
    # with open('coco.names', 'r') as f:
    #     classes = [line.strip() for line in f.readlines()]

    # Load an image for detection
#     img = Image.open('img.jpg')
    img = Image.fromarray(img)
    # Transform the image
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
    #     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img)

    #Perform object detection on the image
    results = model(img)

    # Extract the detected objects and their bounding boxes, class IDs, and confidence scores
    boxes = results.xyxy[0].cpu().numpy()[:, :4]
    class_ids = results.pred[0].cpu().numpy()[:, 5].astype(int)
    confidences = results.pred[0].cpu().numpy()[:, 4]

    # Apply non-max suppression to remove overlapping bounding boxes
    indices = torchvision.ops.nms(torch.from_numpy(boxes), torch.from_numpy(confidences), iou_threshold=0.5)

    # Draw bounding boxes on the image for objects with confidence score above the threshold
    colors = torch.tensor([[0, 255, 0], [0, 0, 255], [255, 0, 0]])
    print(motor_values)
    if len(indices) > 0:
        obstacle_detected = True
        forward_counter = 0
        
        draw = ImageDraw.Draw(img)
        for i in indices:
            x1, y1, x2, y2 = boxes[i]
            label = f'{classes[class_ids[i]]} {confidences[i]:.2f}'
            color = colors[class_ids[i]]
            draw.rectangle([x1, y1, x2, y2], width=2, outline=tuple(color))
            draw.text((x1, y1 - 10), label, fill=tuple(color))
            
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            draw.ellipse([center_x-2, center_y-2, center_x+2, center_y+2], fill=tuple(color), outline=tuple(color))
            
            im_width, im_height = img.size
            draw.ellipse([im_width//2-2, im_height//2-2, im_width//2+2, im_height//2+2], fill=(255, 255, 255), outline=(255, 255, 255))
            error = (center_x - im_width/2) / (im_width/2)
            height = y2 - y1
#             print("height : " + str(height))
#             print("Error : "+str(error))
            
            # Initiate the turn when the bounding box height reaches some threshold.
            if height <=70:
                robot.forward(0.4)
            else:
                robot.stop()
                motor_value = 0.4 +  0.4 * (1 + error)
#                 motor_value= max(0, min(1, motor_value))
                robot.left_motor.value = motor_value
                motor_values.append((robot.left_motor.value, robot.right_motor.value))
    elif len(motor_values) > 0:
        forward_counter += 1
        if forward_counter <= forward_delay:
            robot.left_motor.value = 0.4
            robot.right_motor.value = 0.4
        else: 
            print("Inside else block")
            print(motor_values)
            left_motor, right_motor = motor_values.pop(0)
            print("After pop")
            print(motor_values)
            print("After motor values")
            
            robot.left_motor.value = right_motor
            robot.right_motor.value = left_motor
            time.sleep(0.3)
            
            if len(motor_values) == 0:
                robot.stop()
    else:
        robot.stop()
                
                

            # Add a flag here to initate the turning manuver. 
            # After the obejct is out of the frmae, reverse the order of the commands and get it to the line. 
            # add line following to this. 
    # Display the image with bounding boxes

#     if obstacle_detected == True and len(indices) == 0:
#         robot.forward(0.4)
#         time.sleep(2.5)
#         robot.stop()
#         robot.right_motor.value = 0.5
#         time.sleep(3)
#         obstacle_detected = False
        
        
        
    return np.array(img)